We begin by importing our dataset and printing the head, for an initial exploratory analysis.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import re
import numpy as np
import collections
from collections import Counter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('../input/mbti_1.csv')
print(df.head(10))
print("*"*40)
print(df.info())
print(type(df["type"][[0]][0][0]))


We can thus see that there are no null inputs


In [ ]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
print(df.head())

Since we are dealing with the last 50 posts, we can divide the length of the amount of words by 50 to see an average of words per comment, and we create a new column for it.

In [ ]:
plt.figure(figsize=(15,10))
#sns.swarmplot("type", "words_per_comment", data=df)
sns.violinplot(x='type', y='words_per_comment', data=df, inner=None, color='lightgray')
sns.stripplot(x='type', y='words_per_comment', data=df, size=4, jitter=True)
plt.ylabel('Words per comment')
plt.show()

In [ ]:
df.groupby('type').agg({'words_per_comment':'mean'})

We have constructed a violin plot and checked the mean words per comment, and the second feature does not show anything promising right now. Let's see what else we can do.

In [ ]:
for r in range(len(df['posts'])):
    
    for i in df["type"][[r]]:
        if i[0]=="I":
            df["I-E"][[r]] = 0
        if i[0] == "E":
            df["I-E"] = 1
        if i[1]=="N":
            df["N-S"] = 0
        if i[1] == "S":
            df["N-S"] = 1
        if i[2]=="F":
            df["F-T"] = 0
        if i[2] == "T":
            df["F-T"] = 1
        if i[3]=="J":
            df["J-P"] = 0
        if i[3] == "P":
            df["J-P"] = 1

df['http_per_comment'] = df['posts'].apply(lambda x: x.count('http')/50)
df['music_per_comment'] = df['posts'].apply(lambda x: x.count('music')/50)
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?')/50)
df['img_per_comment'] = df['posts'].apply(lambda x: x.count('jpg')/50)
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!')/50)
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...')/50)
print(df.head(10))

In [ ]:
print(df.groupby('type').agg({'http_per_comment':'mean'}))
print(df.groupby('type').agg({'music_per_comment':'mean'}))
print(df.groupby('type').agg({'question_per_comment':'mean'}))
print(df.groupby('type').agg({'img_per_comment':'mean'}))
print(df.groupby('type').agg({'excl_per_comment':'mean'}))

There's quite a lot of information there. We can do joint plots, pair plots and heat maps to explore relationship between data. 

In [ ]:
plt.figure(figsize=(15,10))
sns.jointplot(x='words_per_comment', y='ellipsis_per_comment', data=df, kind='kde')

So it seems there's a large correlation between words per comment ant the ellipsis the user types per comment!

In [ ]:
i = df['type'].unique()
k = 0
for m in range(0,2):
    for n in range(0,6):
        df_2 = df[df['type'] == i[k]]
        sns.jointplot(x='words_per_comment', y='ellipsis_per_comment', data=df_2, kind="hex")
        plt.title(i[k])
        k+=1


In [ ]:


i = df['type'].unique()
k = 0
TypeArray = []
PearArray=[]
for m in range(0,2):
    for n in range(0,6):
        df_2 = df[df['type'] == i[k]]
        pearsoncoef1=np.corrcoef(x=df_2['words_per_comment'], y=df_2['ellipsis_per_comment'])
        pear=pearsoncoef1[1][0]
        print(pear)
        TypeArray.append(i[k])
        PearArray.append(pear)
        k+=1


TypeArray = [x for _,x in sorted(zip(PearArray,TypeArray))]
PearArray = sorted(PearArray, reverse=True)
print(PearArray)
print(TypeArray)
plt.scatter(TypeArray, PearArray)

Let's do some Machine Learning now!

In [ ]:
X = df.drop(['type','posts'], axis=1).values
y = df['type'].values

print(y.shape)
print(X.shape)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.1, random_state=5)
knn=KNeighborsClassifier(n_neighbors=14)
#knn.fit(X_train,y_train)
#pred=knn.predict(X_test)
#print("accuracy {:.2f}".format(np.mean(pred==y_test)))
#print(knn.score(X_test, y_test))

clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("accuracy {:.2f}".format(np.mean(pred==y_test)))
print(clf.score(X_test, y_test))




So we see our model has an accuracy of 0.20... which is not very good. Let's see what else we can do!

In [ ]:
nova_columna=[]
for z in range(len(df['posts'])):
    prov=df['posts'][z]
    prov2= re.sub(r'[“€â.|,?!)(1234567890:/-]', '', prov)
    prov3 = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', prov)
    prov4 = re.sub(r'[|||)(?.,:1234567890!]',' ',prov3)
    prov5 = re.sub(' +',' ', prov4)
    prov6 = prov5.split(" ")
    counter = Counter(prov6)
    counter2 = counter.most_common(1)[0][0]
    nova_columna.append(counter2)
df['most_used_word'] = nova_columna
print(df.head())
print(df['most_used_word'].unique())